In [42]:
# Importaciones necesarias
import mlflow.pyfunc
from datetime import timedelta
import pandas as pd
import dagshub
import yfinance as yf

In [33]:
# Definir la configuración del modelo y el alias del modelo optimizado
dagshub.init(url="https://dagshub.com/PacoTinoco/Proyecto_Final_CDD", mlflow=True)
MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

model_name = "amazon-stock-model-randomforest-prefect"
alias = "champion"
model_uri = f"models:/{model_name}@{alias}"

# Cargar el modelo entrenado de MLflow
model = mlflow.pyfunc.load_model(model_uri)


Initialized MLflow to track repo "PacoTinoco/Proyecto_Final_CDD"

Repository PacoTinoco/Proyecto_Final_CDD initialized!

In [43]:
# Solicitar datos al usuario
capital_inicial = float(input("Ingrese el capital inicial a invertir en USD: "))
dias_inversion = int(input("Ingrese el número de días de inversión: "))


In [58]:
# Descargar datos históricos recientes para Google
data = yf.download('AMZN', period="1mo")
# Usar la última fila como base
input_data = data.iloc[-1:]

# Iterar para simular la predicción de cada día adicional
precio_inicial = input_data['Close']
cantidad_acciones = capital_inicial / precio_inicial
precio_estimado = precio_inicial


[*********************100%***********************]  1 of 1 completed


In [52]:
input_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-11-04,196.449997,197.330002,194.309998,195.779999,195.779999,38492100
2024-11-05,196.039993,199.820007,195.990005,199.500000,199.500000,30564800
2024-11-06,200.009995,207.550003,199.139999,207.089996,207.089996,72292200
2024-11-07,207.440002,212.250000,207.190002,210.050003,210.050003,52878400
2024-11-08,209.720001,209.960007,207.440002,208.179993,208.179993,36075800
2024-11-11,208.500000,209.649994,205.589996,206.839996,206.839996,35456000
2024-11-12,208.369995,209.539993,206.009995,208.910004,208.910004,38942900
2024-11-13,209.399994,215.089996,209.139999,214.100006,214.100006,46212900
2024-11-14,214.160004,215.899994,210.880005,211.479996,211.479996,42620300


In [53]:
precio_inicial = input_data['Close']
precio_inicial

Date
2024-11-04    195.779999
2024-11-05    199.500000
2024-11-06    207.089996
2024-11-07    210.050003
2024-11-08    208.179993
2024-11-11    206.839996
2024-11-12    208.910004
2024-11-13    214.100006
2024-11-14    211.479996
2024-11-15    202.610001
Name: Close, dtype: float64

In [54]:
for _ in range(dias_inversion):
    # Predecir el precio de cierre del día
    precio_estimado = model.predict(input_data)
    # Actualizar el dataframe de entrada con el precio proyectado
    input_data['Close'] = precio_estimado
    input_data.index += timedelta(days=1)

# Calcular el capital final con la proyección de precio final
capital_final = cantidad_acciones * precio_estimado
ganancia_estimada = capital_final - capital_inicial

C:\Users\Francisco\AppData\Local\Temp\ipykernel_10600\614707743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['Close'] = precio_estimado
C:\Users\Francisco\AppData\Local\Temp\ipykernel_10600\614707743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['Close'] = precio_estimado
C:\Users\Francisco\AppData\Local\Temp\ipykernel_10600\614707743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [55]:
ganancia_estimada    

Date
2024-11-04    -58.074190
2024-11-05    -75.637926
2024-11-06   -109.156036
2024-11-07   -122.065074
2024-11-08   -114.178882
2024-11-11   -108.440161
2024-11-12   -117.274278
2024-11-13   -138.672449
2024-11-14   -128.001525
2024-11-15    -89.464432
Name: Close, dtype: float64

In [56]:
# Extraer valores numéricos para evitar el error de Series
capital_final = capital_final.iloc[0] if isinstance(capital_final, pd.Series) else capital_final
ganancia_estimada = ganancia_estimada.iloc[0] if isinstance(ganancia_estimada, pd.Series) else ganancia_estimada

In [57]:
# Mostrar el resultado
print(f"Se estima que tendrás aproximadamente ${capital_final:.2f} USD al final de {dias_inversion} días, con una ganancia de ${ganancia_estimada:.2f} USD.")

Se estima que tendrás aproximadamente $941.93 USD al final de 10 días, con una ganancia de $-58.07 USD.
